In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import glob
import csv

pd.set_option('display.max_columns', None)

## Load and format NEI industrial data (CAPs and HAPs emissions by county)

In [2]:
nei_point = pd.read_csv('Industrial/EPA NEI/2017neiApr_point_facilitylevel/2017neiApr_point.csv', low_memory=False)
nei_nonpoint = pd.read_csv('Industrial/EPA NEI/2017neiApr_nonpoint/esg_cty_scc_12961.csv', low_memory=False)
scc = pd.read_csv('Industrial/scc_fuelcombustion_oilgas.csv')

In [3]:
states = ['CO', 'NM', 'NV']
dfs = {}

In [4]:
for state in states:
    dfs[state] = []
    point = nei_point[nei_point['state'] == state]
    point['naics code'] = point['naics code'].apply(str)
    point = point[[x.startswith(('21', '2212', '324110', '33313', '333132', '4247', '447', '486')) for x in point['naics code']]]
    point = point[[x.startswith(('2122', '2123', '213114', '213115', '48699', '486990')) == False for x in point['naics code']]]
    dfs[state].append(point)
    
    nonpoint = nei_nonpoint[nei_nonpoint['state'] == state]
    nonpoint = nonpoint[nonpoint['scc'].isin(scc['scc'])]
    
    nonpoint.insert(loc=10, column='Fuel Type', value='N/A')
    fuelcomb_inds = nonpoint['sector'].str.contains('Fuel Comb')
    nonpoint.loc[fuelcomb_inds,'Fuel Type'] = nonpoint['sector'].str.split().str[-1]
    dfs[state].append(nonpoint)
    
    #point.to_csv('Industrial/{}_nei_point_filtered.csv'.format(state))
    #nonpoint.to_csv('Industrial/{}_nei_nonpoint_filtered.csv'.format(state))

/Users/AnnieDillon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [5]:
county_emissions = {}
countybysource_emissions = {}

for state in states:
    point_county = dfs[state][0].groupby(['county','pollutant desc','pollutant type(s)','emissions uom'])['total emissions'].sum().reset_index()
    nonpoint_county = dfs[state][1].groupby(['county','pollutant desc','pollutant type(s)','emissions uom'])['total emissions'].sum().reset_index()
    county_emissions[state] = pd.concat([point_county, nonpoint_county]).groupby(['county','pollutant desc','pollutant type(s)','emissions uom'])['total emissions'].sum().reset_index()
    #county_emissions[state].to_csv('Industrial/{}_industrialemissions_county.csv'.format(state))
    
    point_edit = dfs[state][0].rename(columns={'naics description': 'Source Type'})
    point_edit['data category'] = 'P'
    point_edit.insert(loc=10, column='Fuel Type', value='N/A')
    point_bycountysource = point_edit.groupby(['county','data category','Source Type','Fuel Type','pollutant desc','pollutant type(s)','emissions uom'])['total emissions'].sum().reset_index()
    nonpoint_edit = dfs[state][1].rename(columns={'sector': 'Source Type'})
    nonpoint_bycountysource = nonpoint_edit.groupby(['county','data category','Source Type','Fuel Type','pollutant desc','pollutant type(s)','emissions uom'])['total emissions'].sum().reset_index()
    countybysource_emissions[state] = pd.concat([point_bycountysource, nonpoint_bycountysource], ignore_index=True)
    
    #Reformat dataset for energy equity group analysis. 
    countybysource_emissions[state].insert(loc=0, column='Year',value='2017')
    countybysource_emissions[state].insert(loc=1, column='State',value=state)
    countybysource_emissions[state].columns = [x.title() for x in countybysource_emissions[state].columns]
    countybysource_emissions[state].insert(loc=3, column='Census Tract',value='')
    countybysource_emissions[state].insert(loc=4, column='Sector',value='Industrial')
    countybysource_emissions[state].insert(loc=8, column='Fuel Amount',value='')
    countybysource_emissions[state].insert(loc=9, column='Fuel Units',value='')
    countybysource_emissions[state].rename(columns={'Pollutant Type(S)': 'Pollutant Category'}, inplace=True)
    countybysource_emissions[state].rename(columns={'Pollutant Desc':'Pollutant Type'}, inplace=True)
    countybysource_emissions[state].rename(columns={'Total Emissions':'Annual Emissions'}, inplace=True)
    countybysource_emissions[state].insert(loc=14, column = 'Scenario', value = 'Reference')
    
    ton_inds = countybysource_emissions[state]['Emissions Uom']=='TON'
    lb_inds = countybysource_emissions[state]['Emissions Uom']=='LB'
    countybysource_emissions[state].loc[ton_inds,'Annual Emissions'] = countybysource_emissions[state]['Annual Emissions'] * 0.907185
    countybysource_emissions[state].loc[lb_inds,'Annual Emissions'] = countybysource_emissions[state]['Annual Emissions'] * 0.000453592
    countybysource_emissions[state].drop(columns=['Emissions Uom'], axis=1, inplace=True)
    #countybysource_emissions[state].to_csv('Industrial/Code Outputs/{}_industrialemissions_bycountysource.csv'.format(state))
    

In [6]:
allstates_countybysource_emissions=pd.concat([countybysource_emissions[state] for state in states], ignore_index=True)

## Load EPA GHGRP industrial data (GHG emissions by county)

#### Load and format GHG emissions from industrial combustion

In [7]:
#These datasets need to be assigned to census tracts. Geopandas methods here: https://github.com/geopandas/geopandas/issues/826

states =['Colorado', 'Nevada', 'NewMexico']
mapping = {
    'Colorado':'CO',
    'Nevada':'NV',
    'NewMexico':'NM',
}

combustion_path = r'Industrial/EPA GHGRP/Combustion/*.xls'
combustion_dfs = {}

for state in states:
    combustion_dfs[state]=[]
    concat = pd.DataFrame()
    for file in glob.glob(combustion_path):
        if state in file:
            comb=pd.read_excel(file, skiprows=5)
            file= file.replace('_',' ')
            fueltype = file.split()[-1]
            fueltype = fueltype.replace('.xls','')
            fueltype = fueltype.replace('Products','')
            fueltype = fueltype.replace('Petroleum','Oil')
            fueltype = fueltype.replace('NatGas','Natural gas')
            comb['Fuel Type'] = fueltype
            combustion_dfs[state].append(comb)
    for df in combustion_dfs[state]:
        concat = pd.concat([concat, df])
    combustion_dfs[state]=concat
    #This is where assignment to census tracts should happen.
    
    combustion_dfs[state].columns = [x.title() for x in combustion_dfs[state].columns]
    combustion_dfs[state].rename(columns={'Reporting Year': 'Year','Ghg Quantity (Metric Tons Co2E)':'Annual Emissions'}, inplace=True)
    combustion_dfs[state].insert(loc=1, column='County',value=combustion_dfs[state]['County Name'].str.title())
    #combustion_dfs[state]['County']=combustion_dfs[state]['County'].replace('County','')
    combustion_dfs[state].insert(loc=2, column='Census Tract',value='')
    combustion_dfs[state].insert(loc=3, column='Sector',value='Industrial')
    combustion_dfs[state].insert(loc=4, column='Source Type',value='Fuel Combustion - Industrial')
    combustion_dfs[state].insert(loc=5, column='Data Category',value='P')
    combustion_dfs[state].insert(loc=6, column='Fuel Amount',value='')
    combustion_dfs[state].insert(loc=7, column='Fuel Units',value='')
    combustion_dfs[state].insert(loc=8, column='Pollutant Category', value='GHG')
    combustion_dfs[state].insert(loc=9, column='Pollutant Type', value='CO2e')
    combustion_dfs[state].insert(loc=10, column = 'Scenario', value = 'Reference')
    combustion_dfs[state] = combustion_dfs[state][['Year','State','County', 'Census Tract', 'Sector', 'Source Type',
       'Data Category', 'Fuel Type','Fuel Amount', 'Fuel Units', 'Pollutant Category',
       'Pollutant Type', 'Annual Emissions', 'Scenario','County Name','Subparts','Latitude','Longitude',
       'City Name','Zip Code', 'Parent Companies','Facility Name', 'Ghgrp Id', 'Reported Address']]
    combustion_dfs[state].drop(combustion_dfs[state].columns[14:],axis=1, inplace=True)

combustion_allstates = pd.concat([combustion_dfs[state] for state in states], ignore_index=True)
allstates_countybysource_emissions = pd.concat([allstates_countybysource_emissions,combustion_allstates], ignore_index=True)

/Users/AnnieDillon/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


#### Load and format GHG emissions data from oil/gas/coal sector process emissions

In [8]:
process_path = r'Industrial/EPA GHGRP/Process Emissions/*.xls'
process_dfs = {}

for state in states:
    process_dfs[state]=[]
    concat = pd.DataFrame()
    for file in glob.glob(process_path):
        if state in file:
            process=pd.read_excel(file, skiprows=5)
            file= file.replace('_',' ')
            process['Source Type'] = 'Process Emissions - OilGasCoal'
            process_dfs[state].append(process)
    for df in process_dfs[state]:
        concat = pd.concat([concat, df])
    process_dfs[state]=concat
    #This is where assignment to census tracts should happen.
    
    process_dfs[state].columns = [x.title() for x in process_dfs[state].columns]
    process_dfs[state].rename(columns={'Reporting Year': 'Year','Ghg Quantity (Metric Tons Co2E)':'Annual Emissions'}, inplace=True)
    process_dfs[state].insert(loc=1, column='County',value=process_dfs[state]['County Name'].str.title())
    #combustion_dfs[state]['County']=combustion_dfs[state]['County'].replace('County','')
    process_dfs[state].insert(loc=2, column='Census Tract',value='')
    process_dfs[state].insert(loc=3, column='Sector',value='Industrial')
    process_dfs[state].insert(loc=4, column='Data Category',value='P')
    process_dfs[state].insert(loc=5, column='Fuel Type',value='N/A')
    process_dfs[state].insert(loc=6, column='Fuel Amount',value='N/A')
    process_dfs[state].insert(loc=7, column='Fuel Units',value='N/A')
    process_dfs[state].insert(loc=8, column='Pollutant Category', value='GHG')
    process_dfs[state].insert(loc=9, column='Pollutant Type', value='CO2e')
    process_dfs[state].insert(loc=10, column = 'Scenario', value = 'Reference')
    process_dfs[state] = process_dfs[state][['Year','State','County', 'Census Tract', 'Sector', 'Source Type',
       'Data Category', 'Fuel Type','Fuel Amount', 'Fuel Units', 'Pollutant Category',
       'Pollutant Type', 'Annual Emissions', 'Scenario','County Name','Subparts','Latitude','Longitude',
       'City Name','Zip Code', 'Parent Companies','Facility Name', 'Ghgrp Id', 'Reported Address']]
    process_dfs[state].drop(process_dfs[state].columns[14:],axis=1, inplace=True)

process_allstates = pd.concat([process_dfs[state] for state in states], ignore_index=True)
allstates_countybysource_emissions = pd.concat([allstates_countybysource_emissions,process_allstates], ignore_index=True)

In [9]:
#Still need to add NV and NM transportation data.

co_transportation = pd.read_csv('Transportation/Code Outputs/co_transportationemissions_bycountysource.csv')
co_transportation.drop(columns='Unnamed: 0', inplace=True)

In [10]:
co_transportation.rename(columns={'County Name': 'County'}, inplace=True)

In [11]:
allstates_countybysource_emissions = pd.concat([allstates_countybysource_emissions,co_transportation], ignore_index=True)

In [19]:
#allstates_countybysource_emissions.to_csv('Evolved/countyemissions_transportation_industrial.csv')